# LAADS DAAC Subset-Band-Name Regression Tests

This notebook contains contains a suite of regression tests against LAADS DAAC Subset-Band-Name Harmony Service against reference data generated on premises. 

Subset-Band-name ideally operates on Levels 1B, 2, 3 and 4 data; HDF4 only.

## Prerequisites

The dependencies for this notebook are listed in the environment.yaml. To test or install locally, create the papermill environment used in the automated regression testing suite:

`conda env create -f ./environment.yaml && conda activate papermill-subsetbandname`

A `.netrc` file must also be located in the test directory of this repository.

In [ ]:
## Import shared utility routines:
import sys

sys.path.append('../shared_utils')
from utilities import (
    print_error,
    print_success,
    submit_and_download,
)

from harmony import Client, Collection, Environment, Request

from subset_band_name_utitlities import (
    remove_results_files,
    compare_data,
)

## Set Default Parameters

`papermill` requires default values for parameters used on the workflow. In this case, `harmony_host_url`

The following are the valid values
- Production: https://harmony.earthdata.nasa.gov
- UAT: https://harmony.uat.earthdata.nasa.gov
- SIT: https://harmony.sit.earthdata.nasa.gov
- Local: http://localhost:3000

In [ ]:
harmony_host_url = 'https://harmony.uat.earthdata.nasa.gov'
# harmony_host_url = 'https://harmony.sit.earthdata.nasa.gov'
# harmony_host_url = 'http://localhost:3000'
# harmony_host_url = 'https://harmony.earthdata.nasa.gov'

## Identify Harmony Environment

In [ ]:
host_environment = {
    'http://localhost:3000': Environment.LOCAL,
    'https://harmony.sit.earthdata.nasa.gov': Environment.SIT,
    'https://harmony.uat.earthdata.nasa.gov': Environment.UAT,
    'https://harmony.earthdata.nasa.gov': Environment.PROD,
}


harmony_environment = host_environment.get(harmony_host_url)

if harmony_environment is not None:
    harmony_client = Client(env=harmony_environment)

## Setting up Collection Environment Variables

The cell below sets up the Collection, Granule and other necessary variables for each tested dataset. The datasets provided are in the `UAT` environment. There is currently one dataset for Level 1B.

- Level 1B: MOD021KM
- Variable: EV_250_Aggr500_RefSB

In [ ]:
mod02hkm_non_production_info = {
    'collection': Collection(id='C1260442414-LAADSCDUAT'),
    'granule_id': 'G1261680941-LAADSCDUAT',
    'variable': ['EV_250_Aggr500_RefSB'],
}

mod02hkm_production_info = {
    'collection': Collection(id='C1378577630-LAADS'),
    'granule_id': 'G2796405746-LAADS',
    'variable': ['EV_250_Aggr500_RefSB'],
}

file_indicators = {'MOD02HKM': 'MOD02HKM.EV_250_Aggr500_RefSB_output.hdf'}

reference_data = {
    'MOD02HKM': 'reference_data/MOD02HKM.A2023309.2305.061.pscs_001729112207.hdf'
}

These selected collections and granules are only available in UAT environment.

In [ ]:
mod02hkm_subsetbandname_env = {
    Environment.LOCAL: mod02hkm_non_production_info,
    Environment.UAT: mod02hkm_non_production_info,
    Environment.SIT: mod02hkm_non_production_info,
    # Environment.PROD: mod02hkm_production_info,
}

mod02hkm_subsetbandname_info = mod02hkm_subsetbandname_env.get(harmony_environment)

## Variable Subsetting Test
In the cell below, variable subsetting is tested by subsetting EV_250_Aggr500_RefSB variable from the MOD02HKM granule. The results are then compared against the reference data file.

In [ ]:
mod02hkm_test = True

if mod02hkm_subsetbandname_info is not None:

    mod02hkm_request = Request(
        collection=mod02hkm_subsetbandname_info['collection'],
        granule_id=mod02hkm_subsetbandname_info['granule_id'],
        variables=mod02hkm_subsetbandname_info['variable'],
    )

    submit_and_download(harmony_client, mod02hkm_request, file_indicators['MOD02HKM'])

    if not compare_data(
        reference_data['MOD02HKM'], file_indicators['MOD02HKM'], 'EV_250_Aggr500_RefSB'
    ):
        print_error('MOD02HKM data mismatch.')
        mod02hkm_test = False

    remove_results_files()

## Complete Test Suite With Pass/Fail/Skip

In [ ]:
subsetbandname_tests = mod02hkm_test

if mod02hkm_subsetbandname_info is not None:
    if subsetbandname_tests:
        print_success('Subset-Band-Name test suite PASSED.')
    else:
        raise Exception('Subset-Band-Name test suite FAILED')
else:
    print(
        f'Subset-Band-Name is not configured for this environment: "{harmony_environment}" - skipping tests.'
    )